<a href="https://colab.research.google.com/github/BaconBagel/FDNY-ML/blob/main/MCTS_book.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MCTS based path generation for maximisation of marginal response time reduction potential in NYC**

Monte Carlo tree searches allows for path exploration with adjustable greediness, also known as an exploitation/exploration tradeoff. This is done by weighting the exploration propensity density towards more promising branches, whilst periodically forcing exploration of parent branches even if they do not show promise in the short term. In our model, which simulates a firetruck moving through a grid system with a pre-defined reward matrix, MCTS might be a feasible implementation for the routing problem because it helps avoid the overprioritisation of local maxima whilst still prioritising more promising pathways that can not be explored through a purely stochastic approach due to the high branching factor. 

The first step is to formulate a reward space (the potential response time reduction), an action space (our movement options) and a state space (our grid position). Based on an initial position, at first 4 branches will be created for each grid movement option (up down left right). From these branches, random walks will be performed for a preset future space (x steps from the root node), then we run through these walks matching a pregenerated reward matrix which yields the total reward (this can normalized to a sigmoid between 0 and 1, with 0 representing a loss, and 1 a win). For each branch we then backpropagate the score (normalised win representation) and add it to the respective parent nodes. In the first iteration we will simply add a node to the node that had the highest outcome and generate another random walk from there, and propagate the win back again. Now our algorithm decides the next node to branch from using the equation.

\begin{equation}
UCB = \frac{\ R_i}{n_i} + C \sqrt{\frac{\ln N}{n_i}}
\end{equation}

where $R_i$ is the reward obtained from the $i$th simulation starting from node  $R_i$ is the sum of rewards obtained from all simulations starting from node $i$, and $n_i$ is the number of simulations that have been performed from node $i$. $N$ is the number of simulations run in the parent node. $C$ is the exploration constant, which determines how frequently we return to higher nodes in order to explore their branches to decrease short-sightedness of our approach. $R_i$ can also be seen as the sum of rewards in a randomised simulation from the node $j$.

$R_i=\sum_{i=1}^{n_i} r_i$ 

We also have a discount factor, or reward decay $γ$ that depends on how far out the randomised reward path is. This allows us to take into account the uncertainty ascossiated with the rewards in relation to the branch length. 

\begin{equation}
\sum_{i=1}^{n_i} r_i \gamma^i
\end{equation}
 
The reward is obtained from an encoded tuple that takes into account the grid position and timestep.

Most approaches to Monte Carlo Tree Search usually don’t have a distinction between time steps For example, if two chess players each move their knight back and forth, the event state of the game is identical again, and even though the counts are updated through the backpropagation, the win/lose outcome of the branches should not be altered.
In our model there is sequence/timestep dependence in our reward state and movement, so all of our eventstate data need to be encoded through time. 
Another difference to chess is that we don’t have a win scenario, nor do we have an end of game condition. Instead we have a score that we are trying to maximize, and a cyclical period (days) that we bind our game state to. 

![Image](https://i.imgur.com/0ZTuDQA.png)

*(an angular representation of the tree after firetruck 10,000 simulations with line thickness representing $n_i$ we see that nodes coincide eg. any permutation that has the same content in different order for a given timestep will end in the same expected future reward eg l,l,u,d,r,r,r and r,d,r,r,l,u,l will end up at the same location with identical future rewards, but the rewards in the tree until that pointng coming from the parent nodes might be different)*

**Assumptions and limitations**

Note that we make two assumptions for this model. Firstly, when calculating the expected response time reduction, we assume that the estimated response time is known at dispatch. This is because dispatch/software decide the individual deployment methodology, and secondly, traffic data in New York is reasonably accurate, and all EMS vehicles are capable of communicating their coordinates. Secondly, we assume no actual response in our trained model. The reasoning is that dispatch is not the choice of the vehicle, but policy, and that refusing a deployment shouldn’t play into our decision, as it is not an option in real life. Secondly, taking time out of commission for  response and factors like refueling is not the point of interest of our model. The point of the proposed model is to optimise placement to maximise potential for reduction at dispatch. This is an economic perspective that takes simplifying assumptions in order to allow for more extensive analysis of deployment dynamics. Further limitations can always be added when testing the model. 

[![Video](https://i.imgur.com/BpKQXJO.png))](https://i.imgur.com/zHfxpzF.mp4 "Simulation")

*(illustration of paths taken on the grid byt the simulated firetruck, click on it to see an animation)*

**Initialisation** 

First we import our modules, we use Pandas for data formatting, numpy to make operations translatable to machine code (and eventually GPU compatible to allow for parallel processing of operation, Networkx and Matplotlib are used to visualise our reward development, the tree itself, as well as the path taken by our emergency response vehicle.

We also define a statespace, for this we take 1000 timestepts (which translates to roughly 1.5 minutes in a 24h day) on a 100*100 grid, resulting in 10,000,000 potential states. We also define an action space, which for now is limited to 9, we have left, right, up, down, staying still, as well as double movement to see if travel step sizes alter behaviour. Diagonal movement is not implementd yet, as in NYC diagonal streets are extremely rare, however, in the future Manhattan distance movements will be encoded into the transition function (this will be discussed later. We also define the number of simulations, and create lists for positions and rewards obtained as well as an empty dictionary for path decision to allow us to log every movement.

In [ ]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.collections import LineCollection
from collections import Counter
from matplotlib import rc
import networkx as nx
rc('animation', html='jshtml')

# Define the state space and action space
state_space = 1000 * 100 * 100
action_space = 9
position_history = []
reward_history = []
edges = {}


# Define the number of simulations
num_simulations = 100



For the Monte Carlo tree search, it’s best if we pre-calculate our reward to store it in a matrix, as calculating the reward within each branch would slow everything down significantly. To be able to do this, we need to be able to encode our reward within each timestep. In our simulation, this means we stay at a location for t minutes, match the estimated reduction using our matrix calculation based on the traffic speed and the actual response time as well as the distance from our matrix position to the incident position.For this we need to:

Convert our incident data sheet to a compatible format and encode the coordinates into a single integer that represents our grid position. We can do this by normalizing the ranges of latitude and longitude in our data and scaling them to 100, then rounding to integers. Our first two numbers represent the x coordinates, the last two the y coordinates. So the integer 5062 would represent grid 50, 62. 


Our firetruck deployment data has been matched to traffic and weather for previous mechanisms (regression and the feed forward neural network). For the tree based approach these environmental variables will not play a role in the actual decision mechanism, but will only be implemented by filtering the reward vector generation used within the model. 

For this example, we use deployment data from 1 month (February 2020) with around 20k incident entries, each having a column named:

time	latitude	longitude	snow	temperature	precipitation	response_time	speed

[Link to CSV file](https://drive.google.com/file/d/1qtXjx3AuaziNPH_M0RJLVi95KUllu0LL/view?usp=sharing)

Note that time is already formatted to a normalised value beteen 0 and 1, representing the beginning and end of a 24h day respectively.

To begin, we  our data from a csv into a dataframe. Then we need to convert our latitude and longitude to a 100*100 grid, this is done in the scale_column function. We also convert the time to an integer between 1-1000 to allow for the desired state encoding.


In [ ]:

# Load the CSV file into a pandas dataframe
df = pd.read_csv('feb_neural_data2.csv')


def scale_column(column_name, df):
    # Define the column to be scaled
    column_to_scale = column_name

    # Perform Min-Max scaling to normalize the column between 0 and 1
    min_value = df[column_to_scale].min()
    max_value = df[column_to_scale].max()
    df[column_to_scale] = (df[column_to_scale] - min_value) / (max_value - min_value)

    # Scale the column to be between 0 and 100
    df[column_to_scale] = df[column_to_scale] * 100

    # Convert the column to integer data type
    df[column_to_scale] = df[column_to_scale].astype(int)

    # Print the updated dataframe
    return df

# Multiply the 'x' column by 1000 and convert it to an integer
df['time'] = df['time'] * 1000
df['time'] = df['time'].astype(int)

df = scale_column("latitude", df)
df = scale_column("longitude", df)

df['location_index'] = (df["latitude"] * 100) + df["longitude"]



Next we want to encode our reward. The pivot table matches any overlapping timesteps and encode them into an array we can easily perform a lookup in to obtain the reward for a given grid given the timestep. 

Importantly, this converts our reward and resulting game logic from a sparse to a dense matrix, and will hopefully fight the issues caused by the exponential branching factor increase. We also do the same for traffic speeds so we can obtain information about expected rewards in neighbouring cells in our transition function.

We also create baselines by finding the argmax of the best static spot, and by calculating the maximum reward obtainable if we have no movement limitations. (finding the best grid position in each timestep given no movement limitation, eg a drone travelling at lightspeed).

In [ ]:
columns_to_use = ['time', 'location_index', 'response_time']

# Create a pivot table to aggregate c values with respect to identical (a, b) pairs
pivot_table = df.pivot_table(index=['time', 'location_index'], values='response_time', aggfunc=np.sum)
pivot_table2 = df.pivot_table(index=['time', 'location_index'], values='speed', aggfunc=np.sum)

# Create a 2D numpy array with the specified dimensions and filling in the respective values of c
a_values = np.arange(1, 1001)
b_values = np.arange(1, 10001)
c_values = np.zeros((len(a_values), len(b_values)))
speed_values = np.zeros((len(a_values), len(b_values)))

cycles = 0
for i, a in enumerate(a_values):
    cycles += 1
    print(cycles)
    for j, b in enumerate(b_values):
        if (a, b) in pivot_table.index:
            c_values[i][j] = pivot_table.loc[(a, b), 'response_time']

for i, a in enumerate(a_values):
    cycles += 1
    print(cycles)
    for j, b in enumerate(b_values):
        if (a, b) in pivot_table2.index:
            speed_values[i][j] = pivot_table2.loc[(a, b), 'speed']

random_rewards = np.random.rand(1000, 10000)  # can be ued for testing
rewards = c_values
sum_of_max_values = np.sum(np.amax(c_values, axis=1))
print("Theoretical maximum of a drone travelling a c", sum_of_max_values)
# Calculate the sum of each column
column_sums = np.sum(c_values, axis=0)

# Find the index of the column with the highest sum
max_sum_index = np.argmax(column_sums)

# Sort the column sums in descending order and find the second highest index
sorted_column_sums = np.argsort(column_sums)[::-1]
second_max_sum_index = sorted_column_sums[1]

# Print the result
print("Index with highest sum:", max_sum_index)
print("Second highest index:", second_max_sum_index)



The next step is to define our transition function. This function takes a state and action as an input, and generates the expected reward and next state as an output. For now we have 9 options, representing either standing still, or single/double movements for left, right, up and down. The resulting x and y coordinates are changed as long as they don't exceed the boundaries. 

We also calculate the reward for neighbouring cells.

\begin{align*} 
&reward_{l}, reward_{r}, reward_{u}, reward_{d} = 0, 0, 0, 0\\
&\for n = 1 : \horizon_{reward}-1\\  
&\quad\text{if } n < x < 99-n \text{ and } n < y < 99-n:\\  
&reward_{l} += max(rewards[next_{t}, (x + n)*100 + y] - ((0.4*n)/speed\_{values}[next_{t}, (x+n)*100 + y])*3600, 0)\\
&reward_{r} += max(rewards[next_{t}, (x - n)*100 + y] - ((0.4*n)/speed\_{values}[next_{t}, (x-n)*100 + y])*3600, 0)\\    
&reward_{u} += max(rewards[next_{t}, x*100 + (y + n)] - ((0.4*n)/speed\_{values}[next_{t}, x*100 + (y+n)])*3600, 0)\\
&reward_{d} += max(rewards[next_{t}, x*100 + (y - n)] - ((0.4*n)/speed\_{values}[next_{t}, x*100 + (y-n)])*3600, 0)  
\end{align*}

$n$ represents how far away neighbouring cells on the grid can be. 

$0.4$ Is the approximate length of eah grid/block in kilometres used as an approximation for now

$3600$ Converts the speed from kilometres per hour to seconds. 

After this we take the sum of all of the rewards, this is the response time reduction potential.

We also define a state_index function which converts our grid position and timestep into a unique integer lookup.


In [ ]:
# Define the exploration constant
c = np.sqrt(2)

# Function to transition to next state
def transition(state, action):
    # Unpack the state
    t, x, y = state

    # Compute the next position of the character based on the action
    if action == 0:  # up
        x = max(x - 1, 0)
    elif action == 1:  # down
        x = min(x + 1, 99)
    elif action == 2:  # left
        y = max(y - 1, 0)
    elif action == 3:  # right
        y = min(y + 1, 99)
    if action == 4:  # up
        x = max(x - 2, 0)
    elif action == 5:  # down
        x = min(x + 2, 99)
    elif action == 6:  # left
        y = max(y - 2, 0)
    elif action == 7:  # right
        y = min(y + 2, 99)


    # Compute the next time step
    next_t = min(t + 1, 999)

    # Compute the next state
    next_state = (next_t, x, y)

    # Compute the reward
    reward = rewards[next_t, x * 100 + y]

    # Compute neighbour rewards
    reward_horizon = 0
    reward_l, reward_r, reward_u, reward_d = 0, 0, 0, 0
    if reward_horizon > 0:
        for n in range(1, reward_horizon):
            # check boundaries
            if n < x < 99 - n and n < y < 99 - n:
                reward_l += max(rewards[next_t, (x + n) * 100 + y] - (((0.4*n) / speed_values[next_t, (x + n) * 100 + y]) * 3600), 0)
                reward_r += max(rewards[next_t, (x - n) * 100 + y] - (((0.4*n) / speed_values[next_t, (x - n) * 100 + y]) * 3600), 0)
                reward_u += max(rewards[next_t, x * 100 + (y + n)] - (((0.4*n) / speed_values[next_t, x * 100 + (y + n)]) * 3600), 0)
                reward_d += max(rewards[next_t, x * 100 + (y - n)] - (((0.4*n) / speed_values[next_t, x * 100 + (y - n)]) * 3600), 0)
    reward = reward + reward_l + reward_r + reward_u + reward_d
    return next_state, reward


# Define a function to compute a unique index for each state
def state_index(state):
    t, x, y = state
    return t * 100 * 100 + x * 100 + y



For the actual Monte Carlo tree search, we first initialise two arrays, one for the paths taken previously (N) and one the respective reward outcomes (Q) for each of our generated indexes.

We have one for loop, which runs through the specified number of simulations, and in this for loop, we have two while loops.

The first while loop finds the previously mentioned UCT score of the nodes and propagates down the maximum node until a leaf node is reached, in which case it breaks and goes to the the second loop. The first loop also notes the base reward of the chosen branch to enable the calculation of the best total eward later on.

The second loop iterates a random path until the time step limit is reached in terms of the action space (ie the action space from the root node is above a 1000 movements). The reward takes into account our discount rate which decreases as it is multiplied by the gamma value in each step. If the gamma is 0.5, the fourth step would only have a reward factored by 0.125 of the actual response time reduction. This is why we keep the value relatively high, at around 0.99. It then propagates back the calculated reward to the parent/root nodes the node is connected to. 

Lastly, we propagate back the rewards and update our Q and N arrays, meaning we add the reward to all parent nodes. We also save our reward history and trajectory for logging the performance of the code, this will be key once our bandit grows more arms, and we take multiple trees into account.

The last bit of code if for visualisation of the trees.

In [ ]:
def mcts(state):
    # Initialize the search tree
    N = np.zeros((state_space, action_space))
    Q = np.zeros((state_space, action_space))
    max_reward = 0
    last_update_time = time.time()
    total_time = time.time()
    status = state
    # Run the simulations
    for _ in range(num_simulations):
        if _ % 50 == 0:
            print("SImulation #:", _)
        # Select a path through the tree
        path = []
        current_state = status
        trajectory_positions = []
        reward_base = 0
        while True:
            # Compute a unique index for the current state
            index = state_index(current_state)

            # Compute the UCT values
            UCT = Q[index] + c * np.sqrt(np.log(N[index].sum()) / (N[index] + 1e-9))
            # Select the action with the maximum UCT value
            action = np.argmax(UCT)

            # Add the state-action pair to the path
            path.append((current_state, action))

            # Check if we have reached a leaf node
            if N[index, action] == 0:
                break

            # Transition to the next state
            current_state, init_reward = transition(current_state, action)
            reward_base = 0
            for i in range(len(path)):
                state, action = path[i]
                next_state, rew = transition(state, action)
                reward_base += rew
                if state_index(state) not in edges:
                    edges[state_index(state)] = []
                edges[state_index(state)].append((action, state_index(next_state)))

        # Simulate a trajectory from the leaf node
        total_reward = reward_base
        action_max = len(path)
        gamma = 1
        discount_factor = 1
        while True:
            action_max += 1
            # Select a random action
            action = np.random.randint(action_space)

            # Transition to the next state
            current_state, reward = transition(current_state, action)

            # Update the total reward
            total_reward += discount_factor * reward
            if total_reward > max_reward:
                current_time = time.time()
                max_reward = total_reward
                print('Time elapsed since last update:', current_time-last_update_time)
                print("New maximum reward:", max_reward)
                print("Simulation number", _)
                last_update_time = time.time()

            trajectory_positions.append((current_state[1], current_state[2]))
            # Update the discount factor
            discount_factor *= gamma

            # Check if we have reached a terminal state
            if action_max == 1000:
                break

        # Update the search tree
        for current_state, action in reversed(path):
            index = state_index(current_state)
            N[index, action] += 1
            Q[index, action] += (total_reward - Q[index, action]) / N[index, action]

        position_history.append(trajectory_positions)
        reward_history.append(total_reward + reward_base)
    """
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.set_xlim((0, 100))
    ax.set_ylim((0, 100))
    lines = []
    for state, edge_list in edges.items():
        for action, next_state in edge_list:
            x1, y1 = state % 100, state // 100 % 100
            x2, y2 = next_state % 100, next_state // 100 % 100
            # Calculate the slope of the line
            dx = x2 - x1
            dy = y2 - y1
            slope = dy / dx if dx != 0 else float('inf')
            # Calculate the angle of the line
            angle = np.arctan2(dy, dx)
            # Calculate the coordinates of the intermediate point with a 30-degree angle
            dist = np.sqrt(dx ** 2 + dy ** 2) / 2
            angle += np.deg2rad(30)
            angle_x = x1 + dist * np.cos(angle)
            angle_y = y1 + dist * np.sin(angle)
            # Add the three points to the lines list
            lines.append([(y1, x1), (angle_y, angle_x), (y2, x2)])

    lc = LineCollection(lines, linewidths=1, colors='gray', alpha=0.2)
    ax.add_collection(lc)
    # Customize the plot aesthetics
    ax.set_aspect('equal')
    ax.xaxis.set_ticks([])
    ax.yaxis.set_ticks([])
    ax.set_frame_on(False)
    plt.tight_layout()

    plt.show()"""

    # Return the optimal action
    #return np.argmax(N[state_index])

Lastly, we run the simulation and visualise the output in the form of an animation of our trajectory over time in the different simulation runs, as well as the development of our reward. For now we only have one tree, so we initialise from the center point (50, 50), but initiliasing the starting point will be part of future exploration.

In [ ]:
mcts((0,50,50))


def update(frame):
    # Clear the previous frame
    plt.clf()

    # Plot all previous trajectories with a different color
    for i in range(frame):
        trajectory = position_history[i]
        x_values, y_values = zip(*trajectory)
        plt.plot(x_values, y_values)

fig, ax = plt.subplots()
ax.set_ylim([0, 100])
ax.set_xlim([0, 100])


anim = animation.FuncAnimation(fig, update, frames=len(position_history), blit=False, interval=20)



plt.show()



for trajectory in position_history:
    x_values, y_values = zip(*trajectory)
    plt.plot(x_values, y_values, marker='o', linestyle='-', markersize=1)

plt.xlabel('X')r
plt.ylabel('Y')
plt.title('Movement Visualization')
plt.tight_layout()
plt.show()

# Visualize the learning (Reward history)
plt.plot(reward_history)
plt.xlabel('Simulation')
plt.ylabel('Total Reward')
plt.title('Learning Progress')
plt.show()

# Results

We can plot our expected reward over time given the number of simulations, by doing this we see how the computing time and parameters affect our results.

We see that results improve over time, and that the development is greatly affected by the exploration constant C. However, to know the implications over longer training periods, more simulations will be required.

The "steps" one can see in the plot is from the random walk finding improvements to build from, for now the learning rate seems to be growing, and not diminishing as one might excpect. This shows more iteations are needed to properly explore the path option. at some unknown point in the future the reward will diminish.

The main limits right now are memory and computing time, this can hopefully be addressed by restructuring the code and making it GPU compatible

![Image](https://imgur.com/fdpnhNv.png) *C = sqrt2

![Image](https://i.imgur.com/E3nd7U1.png) *C = 1.5, with 9000 simulations*

![Image](https://imgur.com/a0GGpwY.png)
*C = 1.1*
![Image](https://imgur.com/P4hDRXh.png)
*c = 3*
![Image](https://imgur.com/gomCDUo.png)
*c = 50*

# Next Steps

1. Moving all of the code to numpy and removing list operations to make the code GPU compatible

2. Adding a multi-armed bandit method, running trees in parallel and tuning the (hyper)parameters automatically to increase the learning rate

3. Splitting the method into training and testing sets to test the robustness of the suggested path

4. Splitting the reward matrix depending on environmental conditions